# Gesture Recognition
<b> Use Tensorflow 2.10.0 </b>

## 1. Data Loader and Analysis

In [ ]:
# import library
import os, glob, shutil
import numpy as np
import datetime
import tensorflow as tf
import pandas as pd
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

from keras.layers import Dense, Dropout, Conv3D, Input, MaxPool3D, Flatten, Activation, \
                         TimeDistributed, BatchNormalization, MaxPooling2D, LSTM, GRU
from keras.regularizers import l2
from keras.models import Model, Sequential

# set random seed for whole project
np.random.seed(1)
tf.random.set_seed(1)

In [ ]:
# setting global parameter
data_folder = './data'
checkpoints_folder = './checkpoints'

# setting hyparameter for project
batch_size = 8
num_epochs = 50
dest_size = (128, 128)

### 1.1 Read CSV file

In [ ]:
train_doc = np.random.permutation(open('./data/train.csv').readlines())
val_doc = np.random.permutation(open('./data/val.csv').readlines()) 
train_df = pd.read_csv(os.path.join(data_folder, 'train.csv'), delimiter=';', names=['video', 'label (text)', 'label'])
val_df   = pd.read_csv(os.path.join(data_folder, 'val.csv'), delimiter=';', names=['video', 'label (text)', 'label'])
print('Total train samples :', len(train_df))
print('Total val samples   :', len(val_df))

In [ ]:
# take a look at train and val
print(train_df.head(5))
print(val_df.head(5))

In [ ]:
# create a labels dict with key is numberic and labels is text
label_dict = {
    0 : 'Left Swipe',
    1 : 'Right Swipe',
    2 : 'Stop',
    3 : 'Thumbs Down',
    4 : 'Thumbs Up',
}

### 1.2 Analysis data

In [ ]:
data = train_df.groupby('label').size()
data.plot.bar(xlabel='label', ylabel='count')

In [ ]:
data = val_df.groupby('label').size()
data.plot.bar(xlabel='label', ylabel='count')

In [ ]:
# Size (Height and Width of Data)

Conclusion:
<li> Data is quite balance </li>
<li> Image size is very different between image </li>

### 1.3 Data Generator and Transform
Cause data is left and right sensitive, up and down sensitive, so it would be better to not clip, rotate it in spatial
Cause data is sequence, so we should not random choose the frame order

We can change the contrast and brightness of image
https://docs.opencv.org/3.4/d3/dc1/tutorial_basic_linear_transform.html

In [ ]:
def resize_img(im, desired_size = dest_size[0]):
    """
        Resize image with keep aspect ratio
    """
    old_size = im.shape[:2] # old_size is in (height, width) format

    ratio = float(desired_size)/max(old_size)
    new_size = tuple([int(x*ratio) for x in old_size])

    # new_size should be in (width, height) format

    im = cv2.resize(im, (new_size[1], new_size[0]))

    delta_w = desired_size - new_size[1]
    delta_h = desired_size - new_size[0]
    top, bottom = delta_h//2, delta_h-(delta_h//2)
    left, right = delta_w//2, delta_w-(delta_w//2)

    color = [0, 0, 0]
    new_im = cv2.copyMakeBorder(im, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)
    return new_im

def change_brightness(image):
    alpha = np.random.uniform(0.75, 1.25)
    beta = np.random.uniform(0, 100)
    new_image = np.zeros(image.shape, image.dtype)
    for y in range(image.shape[0]):
        for x in range(image.shape[1]):
            for c in range(image.shape[2]):
                new_image[y,x,c] = np.clip(alpha*image[y,x,c] + beta, 0, 255)
    return new_image

def add_noise(image, threshold = 70):
    """
        Add salt and pepper noise if get random int > threshold
    """
    random_value = np.random.randint(0, 100)
    if random_value > threshold:
        image = image/255.0
        row,col,ch = image.shape
        s_vs_p = 0.5
        amount = 0.004
        out = np.copy(image)
        # Salt mode
        num_salt = np.ceil(amount * image.size * s_vs_p)
        coords = [np.random.randint(0, i - 1, int(num_salt))
              for i in image.shape]
        out[coords] = 1

        # Pepper mode
        num_pepper = np.ceil(amount* image.size * (1. - s_vs_p))
        coords = [np.random.randint(0, i - 1, int(num_pepper))
              for i in image.shape]
        out[coords] = 0
        out = image*255
        return out
    else:
        return image

In [ ]:
from imageio import imread
from skimage.transform import resize

def generator(source_path, folder_list, batch_size, augmentation = False):
    img_idx = [x for x in range(0,30,2)]#create a list of image numbers you want to use for a particular video, we are not using all the 30 images in a video, selected pickup of images
    while True:
        t = np.random.permutation(folder_list)
        num_batches =int(len(t)/batch_size) # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            x=len(img_idx)
            y=dest_size[0]
            z=dest_size[1]
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            cnt_img=0
            for folder in range(batch_size): # iterate over the batch_size
                imgs = sorted(os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0])) # read all the images in the folder
                cnt_img+=1
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = cv2.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item])
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                    if augmentation:
                        image = change_brightness(image)
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    image=resize_img(image)
                    image=image/255.0
                    
                    
                    batch_data[folder,idx,:,:,0] = (image[:,:,0])#normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (image[:,:,1])#normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (image[:,:,2])#normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        cnt_img=0
        batch_cover = num_batches*batch_size
        rem = len(t) - batch_cover
        if(len(t)!=batch_cover):
            batch_data = np.zeros((rem,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((rem,5)) # batch_labels is the one hot representation of the output
            for folder in range(rem):
                imgs = sorted(os.listdir(source_path+'/'+ t[folder + batch_cover].split(';')[0])) # read all the images in the folder
                cnt_img+=1
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in  
                    image = cv2.imread(source_path+'/'+ t[folder + batch_cover].strip().split(';')[0]+'/'+imgs[item])
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    image=resize_img(image)
                    image=image/255.0
                    
                    
                    batch_data[folder,idx,:,:,0] = (image[:,:,0])#normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (image[:,:,1])#normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (image[:,:,2])#normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + batch_size].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

# def generator(source_path, folder_list, batch_size):
#     print( 'Source path = ', source_path, '; batch size =', batch_size)
#     img_idx = [x for x in range(0,30,2)]#create a list of image numbers you want to use for a particular video, we are not using all the 30 images in a video, selected pickup of images
#     while True:
#         t = np.random.permutation(folder_list)
#         num_batches =int(len(t)/batch_size) # calculate the number of batches
#         for batch in range(num_batches): # we iterate over the number of batches
#             x=len(img_idx)
#             y=128
#             z=128
#             batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
#             batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
#             cnt_img=0
#             for folder in range(batch_size): # iterate over the batch_size
#                 imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
#                 cnt_img+=1
#                 for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
#                     image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
#                     #crop the images and resize them. Note that the images are of 2 different shape 
#                     #and the conv3D will throw error if the inputs in a batch have different shapes
#                     norm_image=image/255.0
#                     image_resized=resize(norm_image,(128,128),mode='reflect')
                    
#                     batch_data[folder,idx,:,:,0] = (image_resized[:,:,0])#normalise and feed in the image
#                     batch_data[folder,idx,:,:,1] = (image_resized[:,:,1])#normalise and feed in the image
#                     batch_data[folder,idx,:,:,2] = (image_resized[:,:,2])#normalise and feed in the image
                    
#                 batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
#             yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
#         # write the code for the remaining data points which are left after full batches
#         cnt_img=0
#         batch_cover = num_batches*batch_size
#         rem = len(t) - batch_cover
#         if(len(t)!=batch_cover):
#             for folder in range(rem):
#                 imgs = os.listdir(source_path+'/'+ t[folder + batch_cover].split(';')[0]) # read all the images in the folder
#                 cnt_img+=1
#                 for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
#                     image = imread(source_path+'/'+ t[folder + batch_cover].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
#                     #crop the images and resize them. Note that the images are of 2 different shape 
#                     #and the conv3D will throw error if the inputs in a batch have different shapes
#                     norm_image = image/255.0 # normalizing the image 
#                     image_resized=resize(norm_image, (128,128), mode='reflect')
                    
#                     batch_data[folder,idx,:,:,0] = (image_resized[:,:,0]) #normalise and feed in the image
#                     batch_data[folder,idx,:,:,1] = (image_resized[:,:,1]) #normalise and feed in the image
#                     batch_data[folder,idx,:,:,2] = (image_resized[:,:,2]) #normalise and feed in the image
                    
#                 batch_labels[folder, int(t[folder + batch_size].strip().split(';')[2])] = 1
#             yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

In [ ]:
curr_dt_time = datetime.datetime.now()
train_path = './data/train'
val_path = './data/val'
num_train_sequences = len(train_df)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_df)
print('# validation sequences =', num_val_sequences)
print ('# epochs =', num_epochs)

## Model buiding

In [ ]:
def train_model(model, name, num_epochs = num_epochs, batch_size = batch_size, load_lastest = True):
    # train and val generator
    train_generator = generator(train_path, train_doc, batch_size)
    val_generator = generator(val_path, val_doc, batch_size)
    
    # create folder for saving checkpoints
    model_name = 'model' + '_' + name

    if not os.path.exists(os.path.join(checkpoints_folder, model_name)):
        os.mkdir(os.path.join(checkpoints_folder, model_name))
    
    # optimizers
#     opt = tf.keras.optimizers.Adam(learning_rate=0.001,
#                                    beta_1=0.9,
#                                    beta_2=0.999,
#                                    epsilon=1e-07,
#                                    amsgrad=False,
#                                    name='Adam'
#                                   )
#     model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    optimiser = "Adam" #write your optimizer
    model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    print("###############################################")
    print("Model {} summary".format(name))
    print(model.summary())
    print("###############################################")
    
    # callbacks
    filepath = model_name + '-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'
    checkpoint = tf.keras.callbacks.ModelCheckpoint(os.path.join(checkpoints_folder, model_name, filepath),
                                                                 monitor='val_loss', 
                                                                 verbose=1, 
                                                                 save_best_only=True, 
                                                                 save_weights_only=False, 
                                                                 mode='auto', 
                                                                 period=1) #chekpoints to save model in .h5
    LR = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, cooldown=5, verbose=1,mode='auto',min_delta=0.00001) #learning Rate
    callbacks_list = [checkpoint, LR]
    
    # calculate step_per_epoch
    if (num_train_sequences%batch_size) == 0:
        steps_per_epoch = int(num_train_sequences/batch_size)
    else:
        steps_per_epoch = (num_train_sequences//batch_size) + 1 #in case of remaining data points which are left after full batches

    if (num_val_sequences%batch_size) == 0:
        validation_steps = int(num_val_sequences/batch_size)
    else:
        validation_steps = (num_val_sequences//batch_size) + 1 #in case of remaining data points which are left after full batches

    # Training process
    print("###############################################")
    print("Training model")
    
    init_epoch = 0
    if load_lastest:
        model_lastest_path = sorted(glob.glob(os.path.join(checkpoints_folder, model_name, "*")))
        if len(model_lastest_path) != 0:
            model_lastest_path = model_lastest_path[-1]
            epoch_lastest = int(os.path.basename(model_lastest_path).split('-')[1])
            init_epoch = epoch_lastest
            print("Loading model from epoch:", init_epoch)
            model = tf.keras.models.load_model(model_lastest_path)
    
    history=model.fit(train_generator, 
                      steps_per_epoch=steps_per_epoch, 
                      epochs=num_epochs, verbose=1, 
                      callbacks=callbacks_list, 
                      validation_data=val_generator, 
                      validation_steps=validation_steps, 
                      class_weight=None, 
                      workers=1, 
                      initial_epoch=init_epoch)
    print("###############################################")
    return history

In [ ]:
# plot accuracy & loss
def plot_history(history):
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15,4))
    axes[0].plot(history.history['loss'])   
    axes[0].plot(history.history['val_loss'])
    axes[0].legend(['loss','val_loss'])

    axes[1].plot(history.history['categorical_accuracy'])   
    axes[1].plot(history.history['val_categorical_accuracy'])
    axes[1].legend(['categorical_accuracy','val_categorical_accuracy'])

## 2.1 3D CNN Model

### 2.1.1 C3D
C3D inspired model

In [ ]:
def Model_3D_01(nb_classes):
    """
        C3D Model
    """
    input_shape = (15, 128, 128, 3)
    weight_decay = 0.005
    nb_classes = nb_classes

    inputs = Input(input_shape)
    x = Conv3D(32,(3,3,3),strides=(1,1,1),padding='same',
               activation='relu',kernel_regularizer=l2(weight_decay))(inputs)
    x = MaxPool3D((1,2,2),strides=(1,2,2),padding='same')(x)

    x = Conv3D(64,(3,3,3),strides=(1,1,1),padding='same',
               activation='relu',kernel_regularizer=l2(weight_decay))(x)
    x = MaxPool3D((2,2,2),strides=(2,2,2),padding='same')(x)

    x = Conv3D(64,(3,3,3),strides=(1,1,1),padding='same',
               activation='relu',kernel_regularizer=l2(weight_decay))(x)
    x = MaxPool3D((2,2,2),strides=(2,2,2),padding='same')(x)

    x = Conv3D(128,(3,3,3),strides=(1,1,1),padding='same',
               activation='relu',kernel_regularizer=l2(weight_decay))(x)
    x = MaxPool3D((2,2,2),strides=(2,2,2),padding='same')(x)

    x = Conv3D(128, (3, 3, 3), strides=(1, 1, 1), padding='same',
               activation='relu',kernel_regularizer=l2(weight_decay))(x)
    x = MaxPool3D((2, 2, 2), strides=(2, 2, 2), padding='same')(x)

    x = Flatten()(x)
    x = Dense(1024,activation='relu',kernel_regularizer=l2(weight_decay))(x)
    x = Dropout(0.5)(x)
    x = Dense(128,activation='relu',kernel_regularizer=l2(weight_decay))(x)
    x = Dropout(0.5)(x)
    x = Dense(nb_classes,kernel_regularizer=l2(weight_decay))(x)
    x = Activation('softmax')(x)

    model = Model(inputs, x, name = "3D_01")
    return model

In [ ]:
model = Model_3D_01(nb_classes = 5)
history = train_model(model, name = '3D_01')
plot_history(history)

## 2.2 Model 2
What if 3D CNN go with VGG like (2 conv + 1 pool)

In [ ]:
def Model_3D_02(nb_classes):
    """
        C3D Model
    """
    input_shape = (15, 128, 128, 3)
    weight_decay = 0.005
    nb_classes = nb_classes

    inputs = Input(input_shape)
    # block 1
    x = Conv3D(32,(3,3,3),strides=(1,1,1),padding='same',
               activation='relu',kernel_regularizer=l2(weight_decay))(inputs)
    x = Conv3D(32,(3,3,3),strides=(1,1,1),padding='same',
               activation='relu',kernel_regularizer=l2(weight_decay))(x)
    x = MaxPool3D((1,2,2),strides=(1,2,2),padding='same')(x)
    
    # block 2
    x = Conv3D(64,(3,3,3),strides=(1,1,1),padding='same',
               activation='relu',kernel_regularizer=l2(weight_decay))(x)
    x = Conv3D(64,(3,3,3),strides=(1,1,1),padding='same',
               activation='relu',kernel_regularizer=l2(weight_decay))(x)
    x = MaxPool3D((2,2,2),strides=(2,2,2),padding='same')(x)

    # block 3
    x = Conv3D(64,(3,3,3),strides=(1,1,1),padding='same',
               activation='relu',kernel_regularizer=l2(weight_decay))(x)
    x = Conv3D(64,(3,3,3),strides=(1,1,1),padding='same',
               activation='relu',kernel_regularizer=l2(weight_decay))(x)
    x = MaxPool3D((2,2,2),strides=(2,2,2),padding='same')(x)
    
    # block 4
    x = Conv3D(128,(3,3,3),strides=(1,1,1),padding='same',
               activation='relu',kernel_regularizer=l2(weight_decay))(x)
    x = Conv3D(128,(3,3,3),strides=(1,1,1),padding='same',
               activation='relu',kernel_regularizer=l2(weight_decay))(x)
    x = MaxPool3D((2,2,2),strides=(2,2,2),padding='same')(x)
    
    # block 5
    x = Conv3D(128, (3, 3, 3), strides=(1, 1, 1), padding='same',
               activation='relu',kernel_regularizer=l2(weight_decay))(x)
    x = MaxPool3D((2, 2, 2), strides=(2, 2, 2), padding='same')(x)

    x = Flatten()(x)
    x = Dense(1024,activation='relu',kernel_regularizer=l2(weight_decay))(x)
    x = Dropout(0.5)(x)
    x = Dense(128,activation='relu',kernel_regularizer=l2(weight_decay))(x)
    x = Dropout(0.5)(x)
    x = Dense(nb_classes,kernel_regularizer=l2(weight_decay))(x)
    x = Activation('softmax')(x)

    model = Model(inputs, x, name = '3D_02')
    return model

In [ ]:
model = Model_3D_02(nb_classes = 5)
history = train_model(model, name = '3D_02')
plot_history(history)

## 2.3 Custom CNN + LTSM/GRU

In [ ]:
# 2D + LTSM/GRU 

In [ ]:
# transfer: VGG16/MobileNet/Resnet18/EfficentNet-B0 + LTSM/GRU

In [ ]:
# choose the best => use this idea

## 2.3 Pretrained model with LSTM/GRU

### 2.3.1 VGG16 + GRU

In [ ]:
def Model_Pretrained_01(nb_classes):
    """
        VGG 16 + GRU
    """
    input_shape = (15, 128, 128, 3)
    weight_decay = 0.005
    nb_classes = nb_classes
    
    # load VGG16 and freeze it
    vgg_model = tf.keras.applications.vgg16.VGG16(weights='imagenet', 
                                                  include_top=False, 
                                                  input_shape=(dest_size[0], dest_size[1], 3))
    vgg_model.trainable = False
    

    model = Sequential(name="Pretrained_01")
    model.add(TimeDistributed(vgg_model,input_shape=input_shape))
    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))
    model.add(TimeDistributed(Flatten()))


    model.add(GRU(64, return_sequences=False))
    model.add(Dropout(0.25))

    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.25))

    # using Softmax as last layer
    model.add(Dense(nb_classes, activation='softmax')) 
    
    return model

In [ ]:
model = Model_Pretrained_01(nb_classes = 5)
history = train_model(model, name = 'Pretrained_01')
plot_history(history)

### 2.3.2 ResNet50 + GRU

In [ ]:
def Model_Pretrained_02(nb_classes):
    """
        ResNet50 + GRU
    """
    input_shape = (15, 128, 128, 3)
    weight_decay = 0.005
    nb_classes = nb_classes
    
    # load ResNet50 and freeze it
    resnet50_model = tf.keras.applications.vgg16.VGG16(weights='imagenet', 
                                                       include_top=False, 
                                                       input_shape=(dest_size[0], dest_size[1], 3))
    resnet50_model.trainable = False

    model = Sequential(name="Pretrained_02")
    model.add(TimeDistributed(resnet50_model,input_shape=input_shape))
    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))
    model.add(TimeDistributed(Flatten()))


    model.add(GRU(64, return_sequences=False))
    model.add(Dropout(0.25))

    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.25))

    # using Softmax as last layer
    model.add(Dense(nb_classes, activation='softmax')) 
    
    return model

In [ ]:
model = Model_Pretrained_02(nb_classes = 5)
history = train_model(model, name = 'Pretrained_02')
plot_history(history)

### 2.3.3 EfficientNetv2B0 + GRU

In [ ]:
def Model_Pretrained_03(nb_classes):
    """
        ResNet50 + GRU
    """
    input_shape = (15, 128, 128, 3)
    weight_decay = 0.005
    nb_classes = nb_classes
    
    # load EfficientNetv2B0 and freeze it
    eff_model = tf.keras.applications.efficientnet_v2.EfficientNetV2B0(weights='imagenet', 
                                                                       include_top=False, 
                                                                       input_shape=(dest_size[0], dest_size[1], 3))
    eff_model.trainable = False

    model = Sequential(name="Pretrained_03")
    model.add(TimeDistributed(eff_model,input_shape=input_shape))
    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))
    model.add(TimeDistributed(Flatten()))


    model.add(GRU(64, return_sequences=False))
    model.add(Dropout(0.25))

    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.25))

    # using Softmax as last layer
    model.add(Dense(nb_classes, activation='softmax')) 
    
    return model

In [ ]:
model = Model_Pretrained_03(nb_classes = 5)
history = train_model(model, name = 'Pretrained_03')
plot_history(history)

### 2.3.4 EfficientNetv2B1 + GRU

In [ ]:
def Model_Pretrained_04(nb_classes):
    """
        ResNet50 + GRU
    """
    input_shape = (15, 128, 128, 3)
    weight_decay = 0.005
    nb_classes = nb_classes
    
    # load EfficientNetv2B1 and freeze it
    eff_model = tf.keras.applications.efficientnet_v2.EfficientNetV2B1(weights='imagenet', 
                                                                       include_top=False, 
                                                                       input_shape=(dest_size[0], dest_size[1], 3))
    eff_model.trainable = False

    model = Sequential(name="Pretrained_04")
    model.add(TimeDistributed(eff_model,input_shape=input_shape))
    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))
    model.add(TimeDistributed(Flatten()))


    model.add(GRU(64, return_sequences=False))
    model.add(Dropout(0.25))

    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.25))

    # using Softmax as last layer
    model.add(Dense(nb_classes, activation='softmax')) 
    
    return model

In [ ]:
model = Model_Pretrained_04(nb_classes = 5)
history = train_model(model, name = 'Pretrained_04')
plot_history(history)

### 2.3.5 MobileNetv2 + GRU

In [ ]:
def Model_Pretrained_05(nb_classes):
    """
        MobileNetv2 + GRU
    """
    input_shape = (15, dest_size[0], dest_size[1], 3)
    weight_decay = 0.005
    nb_classes = nb_classes
    
    # load MobileNetv2 and freeze it
    mob_model = tf.keras.applications.mobilenet_v2.MobileNetV2(weights='imagenet',
                                                               include_top=False, 
                                                               input_shape=(dest_size[0], dest_size[1], 3))
    mob_model.trainable = False
    model = Sequential(name="Pretrained_05")
    model.add(TimeDistributed(mob_model,input_shape=input_shape))
    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))
    model.add(TimeDistributed(Flatten()))


    model.add(GRU(64, return_sequences=False))
    model.add(Dropout(0.25))

    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.25))

    # using Softmax as last layer
    model.add(Dense(nb_classes, activation='softmax')) 
    
    return model

In [ ]:
model = Model_Pretrained_05(nb_classes = 5)
history = train_model(model, name = 'Pretrained_05', batch_size=20)
plot_history(history)

In [ ]:
0.98% TRAIN
0.90% VALIDATION

### 2.3.6 Xception + GRU

In [ ]:
def Model_Pretrained_06(nb_classes):
    """
        Xception + GRU
    """
    input_shape = (15, dest_size[0], dest_size[1], 3)
    weight_decay = 0.005
    nb_classes = nb_classes
    
    # load MobileNetv2 and freeze it
    xcep_model = tf.keras.applications.xception.Xception(weights='imagenet',
                                                        include_top=False, 
                                                        input_shape=(dest_size[0], dest_size[1], 3))
    xcep_model.trainable = False 
    
    model = Sequential(name="Pretrained_06")
    model.add(TimeDistributed(xcep_model,input_shape=input_shape))
    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))
    model.add(TimeDistributed(Flatten()))


    model.add(GRU(64, return_sequences=False))
    model.add(Dropout(0.25))

    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.25))

    # using Softmax as last layer
    model.add(Dense(nb_classes, activation='softmax')) 
    
    return model

In [ ]:
model = Model_Pretrained_06(nb_classes = 5)
history = train_model(model, name = 'Pretrained_06', batch_size=8)
plot_history(history)

In [ ]:
def Model_Pretrained_07(nb_classes):
    """
        MobileNet + GRU
    """
    num_featuremaps = [16,32,64,128]
    num_denselayers = [64,32,5]
    num_classes = 5
    frames = 15
    dropout = 0.25
    num_epochs = 30

    # increasing batch Size
    batch_size = 20

    # Input
    input_shape=(15, dest_size[0], dest_size[1], 3)

    # Define model
    mobile_model = tf.keras.applications.mobilenet.MobileNet(weights='imagenet', include_top=False)
    mobile_model.trainable = True

    model = Sequential(name = 'Pretrained_07')
    model.add(TimeDistributed(mobile_model,input_shape=input_shape))

    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))
    model.add(TimeDistributed(Flatten()))

    model.add(GRU(64, return_sequences=False))
    model.add(Dropout(dropout))

    model.add(Dense(num_denselayers[1],activation='relu'))
    model.add(Dropout(dropout))

    # using Softmax as last layer
    model.add(Dense(num_classes, activation='softmax')) 
    return model

In [ ]:
model = Model_Pretrained_07(nb_classes = 5)
history = train_model(model, name = 'Pretrained_07', batch_size=4)
plot_history(history)